## [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [2]:
from imports import *

In [3]:
import os
os.environ["OPENAI_API_KEY"] = API_KEY

In [5]:
# !pip install openai==0.28

In [6]:
import openai
openai.api_key = API_KEY
llm_model = 'gpt-3.5-turbo-0301'

## Helper Function


In [7]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [8]:
#check if it works
#get_completion("What is IITM known for")

'IITM (Indian Institute of Technology Madras) is known for its excellence in engineering education and research. It is one of the premier engineering institutes in India and is known for its rigorous academic curriculum, world-class faculty, state-of-the-art infrastructure, and strong industry collaborations. IITM is also known for its research in various fields such as aerospace, biotechnology, computer science, energy, and materials science. The institute has produced several notable alumni who have made significant contributions to the fields of science, technology, and business.'

## Install LANGCHAIN


In [ ]:
# !pip install --upgrade langchain

# MODEL

ChatOpenAI is an api endpoint integrated with langchain

In [120]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(api_key=API_KEY,temperature=0.0, model=llm_model)

Lets create a template string - this is a basic format string that has input variables text and style

In [121]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

Lets import ChatPromptTemplate - This is a class that helps us create prompt templates in langchain format

In [124]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt
# observe the input_variables and the template as attributes

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [125]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [126]:
# This is a customer email
customer_email = """
Arrr, me blender's gone and blown its top,\
splashing me whole kitchen with smoothie!\
And to top it off, the warranty's no help\
when it comes to cleaning this mess.\
I'm in dire need of yer assistance, me hearty!
"""
# This is the style you want to convert the text to!
customer_style = """American English \
in a calm and respectful tone
"""

In [129]:
# The format_messages method allows one to input the style and text into the prompt
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)
print(type(customer_messages))
print(customer_messages)

<class 'list'>
[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, me blender's gone and blown its top,splashing me whole kitchen with smoothie!And to top it off, the warranty's no helpwhen it comes to cleaning this mess.I'm in dire need of yer assistance, me hearty!\n```\n")]


In [25]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

You write back to the customer but you want it to be in the English pirate slang

In [130]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

Set the style

In [131]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [134]:

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)
service_response = chat(service_messages)
print(service_response.content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


# Output Parsers

Let's start with defining how we would like the LLM output to look like:

## Why ?

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [33]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [34]:
from langchain.prompts import ChatPromptTemplate # Use to create flexible templated prompts for chat models.

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [35]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [36]:
type(response.content)

str

In [37]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary

response.content

'{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}'

In [38]:
response.content.get("gift")

AttributeError: 'str' object has no attribute 'get'

## Parse the LLM output string into a Python dictionary

In [39]:
from langchain.output_parsers import ResponseSchema 
from langchain.output_parsers import StructuredOutputParser #this output parser requires a special output format. 


The response is for the structured output parser to use
It uses the schema to generate a json blob of format instruction

In [40]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]


Generate format instructions

In [137]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [135]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [136]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [138]:
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [139]:
output_dict = output_parser.parse(response.content)
print(output_dict) # output_dict
type(output_dict)

{'gift': True, 'delivery_days': '2', 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


dict

In [140]:
output_dict.get('delivery_days')

'2'

# Memory

In [53]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [54]:
llm = ChatOpenAI(api_key=API_KEY,temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [55]:
conversation.predict(input="Hi, my name is Jaanav")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [56]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [58]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [57]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.


In [59]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [60]:
memory = ConversationBufferMemory()

In [61]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [62]:
print(memory.buffer)

Human: Hi
AI: What's up


In [63]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [64]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [65]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [66]:
from langchain.memory import ConversationBufferWindowMemory

In [67]:
memory = ConversationBufferWindowMemory(k=1)               

In [68]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [69]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [70]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [71]:
conversation.predict(input="Hi, my name is Jaanav")

"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [72]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [73]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory

In [ ]:
#!pip install tiktoken

In [74]:
from langchain.memory import ConversationTokenBufferMemory

llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [75]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [76]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [77]:
from langchain.memory import ConversationSummaryBufferMemory

In [78]:
# create a long string
schedule = "Morning meeting at 9am to review the monthly sales figures. \
Make sure to have your spreadsheet updated with the latest data. \
From 10am-11:30am, you have some time set aside to catch up on emails and organize tasks. \
At 12:30pm, grab lunch with a colleague from the marketing team at the nearby sandwich shop. \
By 2pm, you have a scheduled call with a client to discuss their project requirements. \
You are free from 3pm to 4pm."


memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [79]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in casual conversation before the human asks about their schedule for the day. The AI provides a detailed schedule including a morning meeting, email catch-up time, lunch with a colleague, a client call, and free time in the afternoon.'}

In [80]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [81]:
conversation.predict(input="when can i schedule an unplanned meeting?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in casual conversation before the human asks about their schedule for the day. The AI provides a detailed schedule including a morning meeting, email catch-up time, lunch with a colleague, a client call, and free time in the afternoon.
Human: when can i schedule an unplanned meeting?
AI:

> Finished chain.


'Based on your current schedule, it looks like the best time for an unplanned meeting would be during your free time in the afternoon. Would you like me to block off some time for that?'

In [82]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in casual conversation before the human asks about their schedule for the day. The AI provides a detailed schedule including a morning meeting, email catch-up time, lunch with a colleague, a client call, and free time in the afternoon.\nHuman: when can i schedule an unplanned meeting?\nAI: Based on your current schedule, it looks like the best time for an unplanned meeting would be during your free time in the afternoon. Would you like me to block off some time for that?'}

# Chains


In [83]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [84]:
df.head()

,Product,Review
0,Cotton Bath Towel,"I initially ordered a set of bath towels, but ..."
1,Bluetooth Earbuds,"These earbuds offered excellent sound quality,..."
2,Portable Camping Tent,The tent I received had a minor tear near the ...
3,Throw Blanket,"Among the throw blankets I've tried, this one ..."
4,Electric Juicer,I was thrilled with this juicer initially. How...


In [85]:
from langchain.chains import LLMChain
llm = ChatOpenAI(api_key=API_KEY,temperature=0.9, model=llm_model)

In [86]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [87]:
chain = LLMChain(llm=llm, prompt=prompt)

In [88]:
product = "Cotton Bath Towel"
chain.run(product)

'Cotton Luxe Bath Towels Co.'

## SimpleSequentialChain

In [89]:
from langchain.chains import SimpleSequentialChain

In [90]:
llm = ChatOpenAI(api_key=API_KEY,temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [92]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [93]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [94]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Cotton Luxe Towels" or "Soft Cotton Towels" would be good company names for a company that makes cotton bath towels.
We make luxurious cotton towels for your bath time comfort. Soft and plush, our towels are perfect for everyday use.

> Finished chain.


'We make luxurious cotton towels for your bath time comfort. Soft and plush, our towels are perfect for everyday use.'

## SequentialChain

In [95]:
from langchain.chains import SequentialChain

1. translate the text into english

In [96]:
llm = ChatOpenAI(api_key=API_KEY,temperature=0.9, model=llm_model)
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


2. summarize the review into a single sentence

In [97]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


3. figure out the language of the review to write a reply in the same language

In [98]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


4. prompt the LLM to write a summary in that language

In [99]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


5. combine all the chains together using the sequentialchain class
class SequentialChain(
    *,
    memory: BaseMemory | None = None,
    callbacks: Callbacks = None,
    callback_manager: BaseCallbackManager | None = None,
    verbose: bool = _get_verbosity,
    tags: List[str] | None = None,
    metadata: Dict[str, Any] | None = None,
    chains: List[Chain],
    input_variables: List[str],
    output_variables: List[str],
    return_all: bool = False
)

In [100]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [102]:
review = df.Review[0]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "I initially ordered a set of bath towels, but I wish there was an option for bigger sizes. The towels are soft and absorbent. There was a slight color variation between the towels, but customer service promptly sent replacements. Overall, for the price, I'm content with the purchase.",
 'English_Review': 'Initially, I ordered a set of bath towels but I wish there was an option to buy bigger sizes. The towels are soft and absorbent. There was a slight difference in color between the towels, but the customer service team quickly sent replacements. Overall, for the price, I am happy with my purchase.',
 'summary': 'The bath towels are soft and absorbent but there was a color difference between them, however, the customer service team was helpful and the reviewer is satisfied with their purchase.',
 'followup_message': "Thank you for your feedback on the bath towels. We are thrilled to hear that the towels are soft and absorbent and that our customer service team was helpful in

## Router Chain

In [103]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [104]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Effective for addressing questions related to physics.", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Effective for addressing questions related to math.", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Effective for addressing questions related to History", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Effective for addressing questions related to computer science", 
        "prompt_template": computerscience_template
    }
]

In [105]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [106]:
llm = ChatOpenAI(api_key=API_KEY,temperature=0, model=llm_model)

In [107]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [108]:
destinations_str

'physics: Effective for addressing questions related to physics.\nmath: Effective for addressing questions related to math.\nHistory: Effective for addressing questions related to History\ncomputer science: Effective for addressing questions related to computer science'

In [109]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [110]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [111]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Effective for addressing questions related to physics.
math: Effective for addressing question

In [112]:

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [114]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [115]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/Users/jaanavmathavan/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'Explain black body radiation'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and is described by Planck's law. At low temperatures, the radiation is mostly in the infrared region, while at high temperatures, it shifts towards the visible and ultraviolet regions. The concept of black body radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [116]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


/Users/jaanavmathavan/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'

In [117]:
chain.run("How does a rainbow form?")



> Entering new MultiPromptChain chain...


/Users/jaanavmathavan/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'Explain the physics behind how a rainbow forms.'}
> Finished chain.


'A rainbow forms when sunlight is refracted, or bent, as it enters a raindrop. The light is then reflected off the back of the raindrop and refracted again as it exits the raindrop. This causes the light to separate into its different colors, creating the spectrum of colors we see in a rainbow. The order of the colors, from top to bottom, is red, orange, yellow, green, blue, indigo, and violet. The angle at which the light is refracted and reflected determines the size and shape of the rainbow.'

# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown


In [ ]:
file = '/Users/jaanavmathavan/Desktop/LLMs-workshop/LLM_Shaastra/Langchain/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
from pydantic import ValidationError

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls = DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

## What happened here?


In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0].page_content

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("Hi, my name is Jaanav and I am taking a session on LLMS")
print(embed[:5])

In [ ]:
len(embed)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
docs = text_splitter.split_documents(docs)


In [ ]:
db = DocArrayInMemorySearch.from_documents(
    documents = docs,
    embedding = embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0].page_content

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever,verbose = True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = chain.run(query)

In [ ]:
response

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

# AGENTS

In [ ]:
# pip install langchain_experimental
# !pip install numexpr
# !pip install wikipedia


In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent


In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)


In [ ]:
agent("What is the 25% of 300?")

In [ ]:
agent("which planet has more number of moons, jupyter or saturn?")

In [ ]:
get_completion("Which planet has more moons?")

## Wikipedia example

In [ ]:
question = "what did Guido van Rossum create?"
result = agent(question) 

## Python Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [
                 ["Monica", "Geller"],
                 ["Ross", "Geller"],
                 ["Chandler", "Bing"], 
                 ["Rachel","Green"], 
                 ["Joey","Tribbiani"],
                 ["Phoebe","Buffay"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## Define your own tool

In [ ]:
# !pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")